In [2]:
import math
import tensorflow as tf
import numpy as np
import matplotlib
import os
from tensorflow.contrib import rnn

tf.reset_default_graph()
tf.set_random_seed(777)

if "DISPLAY" not in os.environ:
    matplotlib.use('TkAgg')

import matplotlib.pyplot as plt

def MinMaxScaler(data):
    numerator = data - np.min(data,0)
    denominator = np.max(data,0)-np.min(data,0)
    return numerator / (denominator+1e-7)

xy = np.genfromtxt('/Users/yeseo/Desktop/City_Counted_TaxiMach_Link_Dataset_Full_201501 - 12.txt',delimiter = ',',dtype = None)
xy_with_noise = np.genfromtxt('/Users/yeseo/Desktop/2015eliminated_1.txt',delimiter = ',',dtype = None)

#data_preprocessing
xy= xy[:,:27]
a = xy[:,:2]
b = xy[:,2:]
b= MinMaxScaler(b)
xy = np.hstack((a,b))

xy_with_noise = xy_with_noise[:,:27]
a_with_noise = xy_with_noise[:,:2]
b_with_noise = xy_with_noise[:,2:]
b_with_noise = MinMaxScaler(b_with_noise)
xy_with_noise = np.hstack((a_with_noise,b_with_noise))


#parameters
seq_length =6
data_dim =27
hidden_dim = 25
output_dim = 25
learning_rate = 0.01
iterations = 1500

train_size = int(len(xy)*0.7)
validation_size = int(len(xy)*0.2)

#divide data set to train,validation and test set
train_set = xy[:train_size]
validation_set = xy[train_size:train_size+validation_size]
test_set = xy[train_size+validation_size:]

train_set_with_noise = xy_with_noise[:train_size]
validation_set_with_noise = xy_with_noise[train_size:train_size+validation_size]
test_set_with_noise = xy_with_noise[train_size+validation_size:]

# build data set for rnn
def build_dataset(time_series, seq_length):
    dataX = []
    dataY = []
    for i in range (0,len(time_series)-seq_length):
        _x = time_series[i:i + seq_length, :]
        _y = time_series[i+seq_length,2:]
     
        dataX.append(_x)
        dataY.append(_y)
    return np.array(dataX), np.array(dataY)

#train_set, test_set 만들기
trainX, trainY = build_dataset(train_set,seq_length)
validationX, validationY = build_dataset(validation_set,seq_length)
testX,testY = build_dataset(test_set, seq_length)

trainX_with_noise, trainY_with_noise = build_dataset(train_set_with_noise,seq_length)
validationX_with_noise, validationY_with_noise = build_dataset(validation_set_with_noise,seq_length)
testX_with_noise,testY_with_noise = build_dataset(test_set_with_noise, seq_length)


X1 = tf.placeholder(tf.float32,[None, seq_length,data_dim])
Y1 = tf.placeholder(tf.float32,[None,25])

X2 = tf.placeholder(tf.float32,[None, seq_length,data_dim])
Y2 = tf.placeholder(tf.float32,[None,25])

#LSTM CELL만들기

with tf.variable_scope("rnn1"):
    cell1 = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple = True, activation=tf.tanh)
    outputs1,_states1 = tf.nn.dynamic_rnn(cell1,X1,dtype = tf.float32)
    Y_pred = tf.contrib.layers.fully_connected(outputs1[-1,:], output_dim,activation_fn = None)

    loss1 =tf.reduce_sum(tf.square(Y_pred-Y1))
    train1 = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss1)

with tf.variable_scope("rnn2"):
    cell2 = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple = True, activation=tf.tanh)
    outputs2,_states2 = tf.nn.dynamic_rnn(cell2, X2, dtype = tf.float32)
    Y_pred_with_noise = tf.contrib.layers.fully_connected(outputs2[:,-1], output_dim,activation_fn = None)

    loss2 =tf.reduce_mean(tf.square(Y_pred_with_noise-Y2))
    train2 = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss2)


#RMSE 측정
targets = tf.placeholder(tf.float32,[None,25])
predictions = tf.placeholder(tf.float32,[None,25])
rmse = tf.sqrt(tf.reduce_mean(tf.square(targets-predictions)))

x1 = np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25])
x2 = x1+0.3
x3 = x2+0.3
loss_for_graph = np.zeros(iterations)
x4 = np.array(range(0,iterations))
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(iterations):
        _, step_loss1 = sess.run([train1,loss1],feed_dict={X1:trainX, Y1:trainY})
        print("[step: {}] loss: {}".format(i,step_loss1))
        loss_for_graph[i] = step_loss1
        _, step_loss2 = sess.run([train2,loss2],feed_dict={X2:trainX_with_noise, Y2:trainY_with_noise})
        print("[step: {}] loss: {}".format(i,step_loss2))
        
    test_predict = sess.run(Y_pred, feed_dict = {X1:validationX})
    test_predict_with_noise = sess.run(Y_pred_with_noise, feed_dict = {X2:validationX_with_noise})

    
    rmse_val = sess.run(rmse, feed_dict={targets: validationY,predictions: test_predict})
    print("RMSE: {}".format(rmse_val))
   # print("pred: {}".format(test_predict[-1,:]))
    #print("real: {}".format(testY[-1,:]))
    #print("noise: {}".format(eliminate_noise_pred[-1,:]))
    
#    plt.bar(x1,test_predict[-1,:],label = 'predict',color ='b',width = 0.1)
  #  plt.bar(x2,testY[-1,:],label = 'real',color ='g',width = 0.1)
    #plt.bar(x3,eliminate_noise_pred[-1,:],label = 'noise',color ='g',width = 0.1)
    plt.plot(x4,loss_for_graph)
    #plt.legend()
    plt.show()



    


InvalidArgumentError: Incompatible shapes: [6,25] vs. [2816,25]
	 [[node rnn1/gradients/rnn1/sub_grad/BroadcastGradientArgs (defined at <ipython-input-2-482e0ceecc38>:94)  = BroadcastGradientArgs[T=DT_INT32, _device="/job:localhost/replica:0/task:0/device:CPU:0"](rnn1/gradients/rnn1/sub_grad/Shape, rnn1/gradients/rnn1/sub_grad/Shape_1)]]

Caused by op 'rnn1/gradients/rnn1/sub_grad/BroadcastGradientArgs', defined at:
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_inst

# 